In [2]:
import numpy as np
import os
import cv2
from scipy.optimize import least_squares
import glob

In [3]:
def make_world_grid(pattern_size, square_size):
    """ pattern_size = (cols, rows) of inner corners,
      square_size in your chosen unit (e.g. mm). """
    cols, rows = pattern_size
    objp = np.zeros((cols*rows, 3), dtype=np.float32)
    # x = column index * square_size, y = row index * square_size
    objp[:, :2] = np.mgrid[0:cols, 0:rows].T.reshape(-1, 2) * square_size
    return objp.T  # shape = (3, N)

In [4]:
def auto_detect_corners(img, pattern_size=(4, 4)):
    """
    Try cv2.findChessboardCorners to get Nx2 image points automatically.
    Adjust pattern_size to match your printed template.
    """
    criteria = (cv2.TermCriteria_EPS + cv2.TermCriteria_MAX_ITER, 30, 0.001)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, pattern_size)
    if not ret:
        raise RuntimeError("Chessboard corners not detected. Try a different pattern_size or image.")
    else:
        # Refine the corner positions
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
    #corners = corners.squeeze().T  # 2×N
    return corners2.squeeze().T

In [5]:
def reproj_residuals(P, point3d, point2d):
    P = P.reshape(3,4)
    #  perform calibration and apply it to the 3D points
    points3d_homog = np.hstack((point3d.T, np.ones(( point3d.T.shape[0],1))))
    projected_points = P @ points3d_homog.T
    projected_points = projected_points[:2] / projected_points[2]# Normalize
    pro_2d = projected_points.T
    return  np.linalg.norm(point2d.T - pro_2d, axis=1)


In [6]:
def calibrate(points2d, points3d, Norm = False):
    # Number of points
    if Norm == False:
        points2d = points2d.T
        points3d = points3d.T
    n = points3d.shape[0]

    # Construct the A matrix
    A = []
    for i in range(n):
        X, Y, _ = points3d[i, :]  
        u, v = points2d[i, :]
        X3 = np.array([X,Y,0])
        Y3 = np.array([u,v,0])
        Z  = np.cross(X3, Y3)  
        Z = Z[2]   
        A.append([0, 0, 0, 0, -X, -Y, -Z, -1, v*X, v*Y, v*Z, v])
        A.append([X, Y, Z, 1, 0, 0, 0, 0, -u*X, -u*Y, -u*Z, -u])
        
    A = np.array(A)

    # Perform SVD on A
    _, _, Vt = np.linalg.svd(A )
    M = Vt[-1].reshape(3, 4)
    
    return M

In [7]:
def load_and_detect_all(folder_path, pattern_size=(5, 8), extensions=('png', 'jpg', 'jpeg')):
    # Find all image files in folder with given extensions
    files = []
    for ext in extensions:
        files.extend(glob.glob(os.path.join(folder_path, f'*.{ext}')))
    files = sorted(files)  # optional: sort alphabetically
    point3d = make_world_grid(pattern_size,0.04)
    all_points = []  # list to hold each (2, Ni) array
    all_points3 = []
    for img_path in files:
        img = cv2.imread(img_path)
        if img is None:
            print(f'Warning: failed to load {img_path}')
            continue
        
        # detect corners in this image, returns shape (2, Ni)
        pts = auto_detect_corners(img, pattern_size=pattern_size)
        
        if pts is None or pts.size == 0:
            print(f'No corners detected in {img_path}')
            continue
        else:
            all_points3.append(point3d)
            all_points.append(pts)
            print(f'Detected {pts.shape[1]} points in {os.path.basename(img_path)}')
    
    if not all_points:
        raise RuntimeError("No corner points detected in any image.")
    
    # Concatenate along columns: result is (2, sum(Ni))
    concatenated = np.hstack(all_points)
    concat = np.hstack(all_points3)
    print(f'Total points: {concatenated.shape[1]}')
    return concatenated, concat


In [7]:
point3d = make_world_grid((5,8),0.04)

In [8]:
image_path = "calibration/t_img.png"
img = cv2.imread(image_path)

In [9]:
point2d = auto_detect_corners(img, pattern_size=(5,8))


In [ ]:
folder = 'calibration'
point2d_all, point3d_all = load_and_detect_all(folder, pattern_size=(5, 8))


Detected 40 points in img1.png
Detected 40 points in img2.png
Detected 40 points in t_img.png
Total points: 120


In [105]:
point3d_all.shape

(3, 120)

In [17]:

# Calibrate using the given points
M = calibrate(point2d_all,point3d_all)
print("Projection Matrix M:\n", M)

Projection Matrix M:
 [[-2.94888813e-01 -8.62701310e-01 -2.39181793e-06 -1.35971355e-01]
 [ 2.54365599e-01 -3.65037797e-02  6.25647834e-06 -2.90282675e-01]
 [-4.67232355e-04 -1.52295890e-04  8.48373916e-09 -4.15021682e-04]]


In [31]:
#  perform calibration and apply it to the 3D points
points3d_homog = np.hstack((point3d_all.T[40:80], np.ones(( point3d_all.T[40:80].shape[0],1))))
projected_points = M @ points3d_homog.T
projected_points = projected_points[:2] / projected_points[2]# Normalize
pro_2d = projected_points.T

In [28]:
point2d_all.T[80:].shape

(40, 2)

In [32]:
# calcuate reprojection error and print it
err = np.mean(np.linalg.norm(point2d_all.T[40:80] - pro_2d, axis=1))
print("MSSE")
print("Reprojection error (without normalization):", err)

MSSE
Reprojection error (without normalization): 856.3132448516392


In [12]:
def calibrate_norm(points2d, points3d):
    # implement normalization
    points2d = points2d.T
    points3d = points3d.T
    p_2d = np.hstack((points2d, np.ones((points2d.shape[0], 1)))) 
    p_3d = np.hstack((points3d, np.ones((points3d.shape[0], 1))))  

    mean = np.mean(points2d, axis=0)  
    std = np.std(points2d, axis=0) 
    mean_3d = np.mean(points3d, axis=0)  
    std_3d = np.std(points3d, axis=0)  

    T_2d = np.array([
        [np.sqrt(2) / std[0], 0, -np.sqrt(2) / std[0] * mean[0]],
        [0, np.sqrt(2) / std[1], -np.sqrt(2) / std[1] * mean[1]],
        [0, 0, 1]
    ])
    T_3d = np.array([
        [np.sqrt(3) / std_3d[0], 0, 0, -np.sqrt(3) / std_3d[0] * mean_3d[0]],
        [0, np.sqrt(3) / std_3d[1], 0, -np.sqrt(3) / std_3d[1] * mean_3d[1]],
        [0, 0, np.sqrt(3) / (std_3d[2]+0.0001), -np.sqrt(3) / (std_3d[2]+0.0001) * (mean_3d[2]+0.0001)],
        [0, 0, 0, 1]
    ])

    normalized_2d_pts =  (T_2d @ p_2d.T).T[:, :2]  
    normalized_3d_pts = (T_3d @ p_3d.T).T[:, :3] 
    M = calibrate(normalized_2d_pts, normalized_3d_pts, Norm= True)

    #  do not forget to denormalize

    denormalized_M = np.linalg.inv(T_2d) @ M @ T_3d

    return denormalized_M


In [140]:
# Calibrate with normalization
M_norm = calibrate_norm(point2d_all,point3d_all)
print("Projection Matrix with Normalization:\n", M_norm)

Projection Matrix with Normalization:
 [[ 1.84171504e+03  1.30726996e+03 -5.81857496e+04 -7.95352074e+02]
 [ 1.01333222e+03  6.33599324e+02 -1.26153672e+04 -2.93074114e+02]
 [ 1.67408433e+00  1.13330672e+00 -2.29845533e+01 -4.97515428e-01]]


In [141]:
#  perform calibration and apply it to the 3D points
projected_points = M_norm  @ points3d_homog.T
projected_points = projected_points[:2] / projected_points[2]# Normalize
pro_2d = projected_points.T

In [143]:
# calcuate reprojection error and print it
err = np.mean(np.linalg.norm(point2d_all.T - pro_2d, axis=1))
print("MSSE")
print("Reprojection error (without normalization):", err)

MSSE
Reprojection error (without normalization): 4159.997035147656


In [94]:
def calibrate_opt(points2d, points3d):
    """
    points2d: (N×2) array of image pts
    points3d: (N×3) array of world pts
    returns: 3×4 projection matrix P
    """
    # implement normalization
    points2d = points2d.T
    points3d = points3d.T
    p_2d = np.hstack((points2d, np.ones((points2d.shape[0], 1)))) 
    p_3d = np.hstack((points3d, np.ones((points3d.shape[0], 1))))  

    mean = np.mean(points2d, axis=0)  
    std = np.std(points2d, axis=0) 
    mean_3d = np.mean(points3d, axis=0)  
    std_3d = np.std(points3d, axis=0)  

    T_2d = np.array([
        [np.sqrt(2) / std[0], 0, -np.sqrt(2) / std[0] * mean[0]],
        [0, np.sqrt(2) / std[1], -np.sqrt(2) / std[1] * mean[1]],
        [0, 0, 1]
    ])
    T_3d = np.array([
        [np.sqrt(3) / std_3d[0], 0, 0, -np.sqrt(3) / std_3d[0] * mean_3d[0]],
        [0, np.sqrt(3) / std_3d[1], 0, -np.sqrt(3) / std_3d[1] * mean_3d[1]],
        [0, 0, np.sqrt(3) / (std_3d[2]+0.0001), -np.sqrt(3) / (std_3d[2]+0.0001) * (mean_3d[2]+0.0001)],
        [0, 0, 0, 1]
    ])

    normalized_2d_pts =  (T_2d @ p_2d.T).T[:, :2]  
    normalized_3d_pts = (T_3d @ p_3d.T).T[:, :3] 
    M = calibrate(normalized_2d_pts, normalized_3d_pts, Norm= True)
    DM = np.linalg.inv(T_2d) @ M @ T_3d 
    # initial guess x0 = P0.flatten()
    DM = DM .flatten()
    res = least_squares(reproj_residuals, DM, args=(points3d.T, points2d.T),method='lm')   # Levenberg–Marquardt or Gauss–Newton
    P_opt = res.x.reshape(3,4)

    return P_opt
   

### Final Calibration

In [ ]:
def calibrate(imgs):
    """Calibrate the camera using images of a scene.

    Args:
        imgs (list<PIL.Image>): a list of PIL images to be used for calibration

    Returns:
        results of calibration that could be used for finding 3D positions of robot, blocks and target positions.
        They could, for example, contain camera frame, projection matrix, etc.
    """    
    pattern_size =(6,8)
    square_size = 0.04
    # implement normalization
    cols, rows = pattern_size
    objp = np.zeros((cols*rows, 3), dtype=np.float32)
    # x = column index * square_size, y = row index * square_size
    objp[:, :2] = np.mgrid[0:cols, 0:rows].T.reshape(-1, 2) * square_size

    criteria = (cv2.TermCriteria_EPS + cv2.TermCriteria_MAX_ITER, 30, 0.001)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, pattern_size)
    if not ret:
        raise RuntimeError("Chessboard corners not detected. Try a different pattern_size or image.")
    else:
        # Refine the corner positions
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
    #corners = corners.squeeze().T  # 2×N
   
    
    points2d = corners2.squeeze()
    points3d = objp
    p_2d = np.hstack((points2d, np.ones((points2d.shape[0], 1)))) 
    p_3d = np.hstack((points3d, np.ones((points3d.shape[0], 1))))  

    mean = np.mean(points2d, axis=0)  
    std = np.std(points2d, axis=0) 
    mean_3d = np.mean(points3d, axis=0)  
    std_3d = np.std(points3d, axis=0)  

    T_2d = np.array([
        [np.sqrt(2) / std[0], 0, -np.sqrt(2) / std[0] * mean[0]],
        [0, np.sqrt(2) / std[1], -np.sqrt(2) / std[1] * mean[1]],
        [0, 0, 1]
    ])
    T_3d = np.array([
        [np.sqrt(3) / std_3d[0], 0, 0, -np.sqrt(3) / std_3d[0] * mean_3d[0]],
        [0, np.sqrt(3) / std_3d[1], 0, -np.sqrt(3) / std_3d[1] * mean_3d[1]],
        [0, 0, np.sqrt(3) / (std_3d[2]+0.0001), -np.sqrt(3) / (std_3d[2]+0.0001) * (mean_3d[2]+0.0001)],
        [0, 0, 0, 1]
    ])

    normalized_2d_pts =  (T_2d @ p_2d.T).T[:, :2]  
    normalized_3d_pts = (T_3d @ p_3d.T).T[:, :3] 
    M = calibrate(normalized_2d_pts, normalized_3d_pts, Norm= True)
    DM = np.linalg.inv(T_2d) @ M @ T_3d 
    # initial guess x0 = P0.flatten()
    DM = DM .flatten()
    res = least_squares(reproj_residuals, DM, args=(points3d.T, points2d.T),method='lm')   # Levenberg–Marquardt or Gauss–Newton
    P_opt = res.x.reshape(3,4)

    return P_opt

In [144]:
# Calibrate with optimization
M_opt = calibrate_opt(point2d_all,point3d_all)
print("Projection Matrix with Normalization:\n", M_opt)

Projection Matrix with Normalization:
 [[ 1.87198670e+03  1.34792933e+03 -5.81857496e+04 -5.70766225e+02]
 [ 1.02471171e+03  6.89654766e+02 -1.26153672e+04 -3.02764170e+02]
 [ 2.02397236e+00  1.52042114e+00 -2.29845533e+01 -6.29730139e-01]]


In [145]:
#  perform calibration and apply it to the 3D points
projected_points = M_opt  @ points3d_homog.T
projected_points = projected_points[:2] / projected_points[2]# Normalize
pro_2d = projected_points.T

In [146]:
# calcuate reprojection error and print it
err = np.mean(np.linalg.norm(point2d_all.T - pro_2d, axis=1))
print("MSSE")
print("Reprojection error (without normalization):", err)

MSSE
Reprojection error (without normalization): 412.3400565831396
